In [13]:
import pandas as pd
import numpy as np
import math
import scipy.optimize as opt
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [28]:
FIXED_DATA = ['3u_fixed_x_1.txt', '3mf_I=74.9_A=3_f=.5_tot=2.txt']
FLOATING_DATA = ['3ml_I=744.9_r=5000_tot=1.txt', '3ml_I=744.9_r=5000_tot=1 2.txt']

def load_data(file):
    return pd.read_csv(file, delimiter='\t', names=['qpdx', 'qpdy', 'sgx', 'sgy'])

load_data('opttrap/' + FLOATING_DATA[0])

,a,b,c,d
0,0.008764,-0.018505,5.12189,5.09593
1,0.009421,-0.019162,5.12222,5.09626
2,0.009421,-0.018834,5.12123,5.09429
3,0.009749,-0.019162,5.12156,5.09133
4,0.010078,-0.019162,5.12090,5.09166
5,0.010406,-0.019491,5.12090,5.09363
6,0.011392,-0.019162,5.11893,5.09429
7,0.011721,-0.019491,5.11762,5.09298
8,0.012049,-0.019491,5.11893,5.09396
9,0.012049,-0.019819,5.11959,5.09495
